In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0

# Demonstration of tool call with Amazon Nova models using Bedrock APIs: converse and invoke


### Install the requirements

In [ ]:
! pip install -r ../requirements.txt

### Import the required packages

In [ ]:
import boto3
from botocore.config import Config
import json
import logging
from enum import Enum
from tqdm import tqdm


### Setup tools

To properly train our model on tool usage we need to define our tool definitions. We can do so by defining functions with explicit typed inputs and structured docstrings. 

We are going to define 8 tools:
- weather_api_call
- stat_pull
- text_to_sql
- terminal
- wikipedia
- duckduckgo_results_json
- youtube_search
- pubmed_search

While we are defining 8 tools, we are only going to train our model on 7 of them . This is so that we can test out our performance on unseen tools after training (weather_api_call is the held out tool).

In [ ]:
import weather_api_call, stat_pull,terminal,text_to_sql,wikipidea,youtube_search, pubmed_search, duckduckgo_results_json


### Connecting to Bedrock 

In [ ]:
# setup bedrock and define model for inference
model_id = "amazon.nova-micro-v1:0"  # change the model id depending on what you want to use
my_config = Config(
    region_name = 'us-east-1',
    retries = {
        'max_attempts': 5,
        'mode': 'standard'
    }
)
bedrock = boto3.clisent(service_name="bedrock", config=my_config)
bedrock_runtime = boto3.client(service_name="bedrock-runtime", config=my_config)


#define the messages
sys_msg ="""You are a bot that can handle different requests with tools."""
system_prompt = [{"text": sys_msg}]

question = "Hey, what's the temperature in Paris right now?"
# Next, create a chat and apply the chat template

messages = [
  {"role": "user", "content": [{"text":question}]},
]


# Prepare the tool configuration with the weather tool's specification
tool_config = {"tools": [weather_api_call.get_tool_spec(),
                         stat_pull.get_tool_spec(),
                         terminal.get_tool_spec(),
                         text_to_sql.get_tool_spec(),
                         wikipidea.get_tool_spec(),
                         youtube_search.get_tool_spec(),
                         pubmed_search.get_tool_spec(),
                         duckduckgo_results_json.get_tool_spec()                        
                        ]
              }



In [ ]:
tool_config

## Make inference with converse api

In [ ]:
response = bedrock_runtime.converse(
            modelId=model_id,
            messages=messages,
            system=system_prompt,
            toolConfig=tool_config,
        )

In [ ]:
response

In [ ]:
for content_block in response['output']['message']["content"]:
    
    if "toolUse" in content_block:
            out_tool_name=content_block['toolUse']['name']
            out_tool_inputs_dict=content_block['toolUse']['input']
            print(out_tool_name,out_tool_inputs_dict.keys())

## Make inference with invoke api

In [ ]:
# appropriate prompt template for tool calling 

promt_template = """
Given the following functions within <tools>, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.
Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables. Donot give any explanations. 
ONLY output the resulting JSON structure and nothing else.Donot use the word 'json' anywhere in the result.

<tools>{tool_config}</tools>

Generate answer for the following question.
<question>{question}</question>
"""
# Convert tools configuration to JSON string
formatted_tool_config = json.dumps(tool_config, indent=2)


In [ ]:
question = "Create a new directory named 'project' one level below the current folder"
# Convert tools configuration to JSON string
formatted_tool_config = json.dumps(tool_config, indent=2)

prompt = promt_template.replace("{question}", question)
prompt = prompt.replace("{tool_config}", formatted_tool_config)

# message template
messages = [
        {
            "role": "user",
            "content": [
                {
                   "text": prompt
                }
            ]
        }
    ]

max_tokens= 4096
temperature= 0.2
inferenceConfig = {
                "max_new_tokens": max_tokens,
                "temperature": temperature, 
                # "top_p": float,
                # "top_k": 1
            }

# Prepare request body
model_kwargs = {"system":system_prompt,
                "messages": messages,
                 "inferenceConfig": inferenceConfig,}

body = json.dumps(model_kwargs)

accept = "application/json"
contentType = "application/json"

# invoke the model to make inference
response = bedrock_runtime.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=contentType
    )

response_body = json.loads(response.get("body").read())
# Parse response
response_text = response_body['output']['message']['content'][0]['text']
#response_text = response_body['content'][0]['text']


In [ ]:
response_text.strip('\n')